In [226]:
import json
import pandas as pd
import re
import numpy as np
import tqdm
from math import sqrt, floor
import random

### json 파일 불러오기

In [15]:
data = []
with open('..\\데이터\\Epinion\\epinions_2.json', 'r', encoding='UTF-8') as f:
    for line in f:
        line = json.dumps(line)
        data.append(json.loads(line))
item = pd.DataFrame(data)

In [16]:
temp = []
for index,value in item.iterrows():
    temp.append(json.loads(value[0].replace("'","\"")))

In [17]:
df = pd.DataFrame(temp)

## pickle로 저장

In [9]:
import pickle 
with open('../데이터/Epinion/epinions_2.pickle', 'wb') as f:
    pickle.dump(df, f)   

In [10]:
df.head()

,user,stars,time,item,review
0,fgb59h,4.0,1027296000,Minolta_QMS_PagePro_1250E_Printers,i got this printer from minolta as a warranty ...
1,bucho_ky,2.0,1201305600,Sony_VAIO_PCG_K45_P4_538_3_2GHz_1MB_L2_533MHz_...,although i love my vaio especially the gorgeou...
2,redp944,4.0,1118016000,Sony_VAIO_PCG_K45_P4_538_3_2GHz_1MB_L2_533MHz_...,this is my first laptop and i bought it about ...
3,spongebag7,4.0,1149292800,pr-Durabrand_CD-85_Personal_CD_Player,just recently my sony dej001 walkman died on m...
4,pyros7,5.0,1012262400,pr-Sony_KV_36FS12__Standard_Televisions,well when i graduated from college it didnt ta...


In [11]:
print("참여하는 유저의 수 : ", len(df['user'].unique()))
print("참여하는 item의 수 : ", len(df['item'].unique()))
print("interaction의 수 : ", len(df))

참여하는 유저의 수 :  116260
참여하는 item의 수 :  41269
interaction의 수 :  188478


In [151]:
import pickle
with open('..\\데이터\\Epinion\\epinions_2.pickle', 'rb') as f:
    df = pickle.load(f)

In [20]:
trust = pd.read_table('..\\데이터\\Epinion\\network_trust.txt', sep = ' ', header=None,names=['give', 'trust', 'take'])

In [21]:
trust.head()

,give,trust,take
0,lavenderfruit,trust,kristinafh
1,shippo225,trust,sallyg8
2,aryeh7000,trust,liteair
3,chiquita,trust,magenta321
4,jeffsullivan,trust,rpllingrock


In [219]:
trust_user = set()
trust_user.update(trust['give'].unique())
trust_user.update(trust['take'].unique())

In [ ]:
print(len(trust_user))

In [23]:
review_user = set(df['user'].unique())

In [24]:
print('review를 남기지 않은 user의 수 : ' ,len(trust_user-review_user))
print('link가 없는 user의 수 : ', len(review_user-trust_user))

review를 남기지 않은 user의 수 :  66916
link가 없는 user의 수 :  98645


In [217]:
user_set = set()
user_set.update(trust['give'].unique())
user_set.update(trust['take'].unique())
user_set.update(df['user'].unique())

In [94]:
df.head()

,user,stars,time,item,review
0,fgb59h,4.0,1027296000,Minolta_QMS_PagePro_1250E_Printers,i got this printer from minolta as a warranty ...
1,bucho_ky,2.0,1201305600,Sony_VAIO_PCG_K45_P4_538_3_2GHz_1MB_L2_533MHz_...,although i love my vaio especially the gorgeou...
2,redp944,4.0,1118016000,Sony_VAIO_PCG_K45_P4_538_3_2GHz_1MB_L2_533MHz_...,this is my first laptop and i bought it about ...
3,spongebag7,4.0,1149292800,pr-Durabrand_CD-85_Personal_CD_Player,just recently my sony dej001 walkman died on m...
4,pyros7,5.0,1012262400,pr-Sony_KV_36FS12__Standard_Televisions,well when i graduated from college it didnt ta...


## user에 index를 부여하고 저장

In [152]:
unique_user = set(df['user'])
map_user = []
delete_user = []
index = 0
for user in tqdm.tqdm(unique_user,'set'):
    match_user = df[df['user']==user]
    if len(match_user) > 1:
        map_user.append([user, index])
        index+=1
    else:
        delete_user.append(user)
        df.drop(match_user.index, inplace=True)
map_user_df = pd.DataFrame(map_user, columns = ['org_id', 'remap_id'])

set: 100%|██████████| 116260/116260 [1:08:09<00:00, 28.43it/s]


In [153]:
print("review를 남긴 user의 Unique 수", len(map_user))

review를 남긴 user의 Unique 수 27545


In [154]:
map_user_df.to_csv('./Epinion/user_review_list.txt', sep = '\t', index =False)

In [156]:
trust_user_index = []
map_user_set = set(map_user_df['org_id'])
for t in tqdm.tqdm(trust_user, 'set'):
    if t not in map_user_set:
        trust_user_index.append([t, index])
        index+=1
trust_user_df = pd.DataFrame(trust_user_index, columns = ['org_id', 'remap_id'])

set: 100%|██████████| 84531/84531 [00:00<00:00, 434121.97it/s]


In [157]:
map_user_df_2 = pd.concat([map_user_df, trust_user_df], ignore_index=True)

In [214]:
len(map_user_df_2)

102804

In [158]:
map_user_df_2.to_csv('./Epinion/user_list.txt', sep = '\t', index =False)

## item에 index를 부여 

In [191]:
map_list = []
item_list = df['item'].unique()
delete_item = []
index = 0
for item in tqdm.tqdm(item_list, desc='list'):
        map_list.append([item, index])
        index+=1
map_df = pd.DataFrame(map_list, columns=['org_id', 'remap_id'])

list: 100%|██████████| 37279/37279 [00:00<00:00, 37618.67it/s]


In [193]:
map_df.to_csv('./Epinion/item_list.txt', sep = '\t', index = False)

In [160]:
map_user_dict = dict(zip(map_user_df.org_id,map_user_df.remap_id))

In [194]:
map_item_dict = dict(zip(map_df.org_id,map_df.remap_id))
map_id_item_dict = dict(zip(map_df.remap_id, map_df.org_id))

In [203]:
u_i_rating = df[['user', 'item']]

In [204]:
u_i_rating.head()

,user,item
0,fgb59h,Minolta_QMS_PagePro_1250E_Printers
3,spongebag7,pr-Durabrand_CD-85_Personal_CD_Player
4,pyros7,pr-Sony_KV_36FS12__Standard_Televisions
5,daleb,pr-Sony_KV_36FS12__Standard_Televisions
7,genkle1,pr-Sony_KV_36FS12__Standard_Televisions


In [205]:
u_i_rating['user'] = u_i_rating['user'].map(lambda x: map_user_dict[x])
u_i_rating['item'] = u_i_rating['item'].map(lambda x: map_item_dict[x])

ipykernel_launcher:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ipykernel_launcher:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [206]:
u_i_rating = np.asarray(u_i_rating)

In [199]:
interaction = []
map_user_ar = sorted(map_user_dict.values())
for user_id in map_user_ar:
    match = u_i_rating[u_i_rating[:,0] == user_id]
    interaction.append([user_id]+list(match[:,1]))

In [200]:
item_set = set()
for l in interaction:
    item_set.update(l[1:])

In [222]:
print("Edge에 참여하고 Review를 남긴 Unique user의 수 : ", len(map_user_df_2))
print("Edge에만 참여하는 Unique user의 수 : ", len(trust_user))
print("Review만 남긴 Unique user의 수 : ", 116260)
print()
print("review를 2개이상 남긴 user의 Unique 수", len(map_user))
print("Unique item의 개수 : ",len(item_set))
print()
print("전처리하기 전의 Unique item의 수 : ",41269)
print()



Edge에 참여하고 Review를 남긴 Unique user의 수 :  102804
Edge에만 참여하는 Unique user의 수 :  84531
Review만 남긴 Unique user의 수 :  116260

review를 2개이상 남긴 user의 Unique 수 27545
Unique item의 개수 :  37279

전처리하기 전의 Unique item의 수 :  41269



## totla_data_2.txt는 index 조절한 것

In [209]:
with open('./Epinion/total_data_2.txt', 'w') as output:
    for line in interaction:
        output.write(" ".join(map(str,line)))
        output.write('\n')

## train(0.8)과 test(0.2) 나누기 

In [229]:
train_set = {}
test_set = {}
with open('./Epinion/total_data_2.txt', 'r') as output:
    for l in output.readlines():
        l = l.strip('\n').split(' ')
        items = [int(i) for i in l[1:]]
        uid = int(l[0])
        train_length = floor(len(items)*0.8)
        train_set[uid] = random.sample(items, train_length)
        test_set[uid] = list(set(items) - set(train_set[uid]))
    

In [230]:
with open('./Epinion/train.txt', 'w') as output:
    for k, v in train_set.items():
        temp_list = [k]+v
        output.write(" ".join(map(str, temp_list)))
        output.write('\n')
with open('./Epinion/test.txt', 'w') as output:
    for k, v in test_set.items():
        temp_list = [k]+v
        output.write(" ".join(map(str, temp_list)))
        output.write('\n')